In [2]:
from seldon_deploy_sdk import Configuration, ApiClient, SeldonDeploymentsApi, ModelMetadataServiceApi, DriftDetectorApi, BatchJobsApi, BatchJobDefinition
from seldon_deploy_sdk.auth import OIDCAuthenticator

In [3]:
SD_IP = "34.91.135.63"
config = Configuration()
config.host = f"http://{SD_IP}/seldon-deploy/api/v1alpha1"
config.oidc_client_id = "sd-api"
config.oidc_server = f"http://{SD_IP}/auth/realms/deploy-realm"
config.oidc_client_secret = "sd-api-secret"
config.auth_method = "client_credentials"


def auth():
    auth = OIDCAuthenticator(config)
    config.id_token = auth.authenticate()
    api_client = ApiClient(configuration=config, authenticator=auth)
    return api_client

In [5]:
deployment = {'apiVersion': 'machinelearning.seldon.io/v1alpha2',
 'kind': 'SeldonDeployment',
 'metadata': {'labels': {'fluentd': 'true'},
  'name': 'fraud-andrew-josh',
  'namespace': 'seldon-demos'},
 'spec': {'annotations': {'seldon.io/engine-seldon-log-messages-externally': 'true'},
  'name': 'fraud',
  'predictors': [{'componentSpecs': [{'spec': {'containers': [{'name': 'fraud-container',
         'resources': {'limits': {'cpu': '1', 'memory': '1Gi'},
          'requests': {'cpu': '1', 'memory': '1Gi'}}}]}}],
    'graph': {'children': [],
     'endpoint': {'type': 'REST'},
     'implementation': 'XGBOOST_SERVER',
     'logger': {'mode': 'all'},
     'modelUri': 'gs://tom-seldon-examples/fraud-workshop/models/josh-pretrained',
     'name': 'fraud-container',
     'parameters': []},
    'name': 'default',
    'replicas': 1,
    'traffic': 100}],
  'protocol': 'seldon',
  'transport': 'rest'},
 'status': {}}

In [7]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.create_seldon_deployment(namespace="seldon-demos", mldeployment=deployment)

## Check Status of Cluster

In [ ]:
try:
    api_response = deployment_api.validate_seldon_deployment(namespace, mldeployment)
    pprint(api_response)
except ApiException as e:
    print("Exception when calling SeldonDeploymentsApi->validate_seldon_deployment: %s\n" % e)